In [2]:
str_code= """
from typing import TypedDict, List, Dict, Any, Literal
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_openai import ChatOpenAI
from langgraph.types import Command
import os
from langchain_community.utilities.cassandra_database import CassandraDatabase


# Define Message and MessagesState types
class Message(TypedDict, total=False):
    role: str
    sender: str
    content: str
    timestamp: str
    escalated: bool
    # Additional fields can be added as needed, e.g. 'status', 'agent_id', etc.

# --- supervisor_agent node ---
def supervisor_agent(state: MessagesState) -> Command[Literal["customer_info_agent", "product_info_agent", "requirements_agent", "product_suggestion_agent", "__end__"]]:
    \"\"\"A supervisor managing multiple agents for customer service tasks.\"\"\"
    messages = state.get("messages", [])

    escalation_keywords = {"complaint", "not working", "issue", "problem", "escalate"}
    escalated = False

    for msg in messages:
        content_lower = msg["content"].lower()
        if any(keyword in content_lower for keyword in escalation_keywords):
            if not msg.get("escalated", False):
                msg["escalated"] = True
                escalated = True

    if escalated:
        supervisor_message: Message = {
            "sender": "supervisor",
            "content": "Issue escalated. Assigning to senior agent for immediate attention.",
            "timestamp": "2024-06-01T12:00:00Z"  # In real case, use current timestamp
        }
        messages.append(supervisor_message)

    # Routing logic based on message contents
    wants_customer_info = any("update customer info" in msg["content"].lower() for msg in messages)
    wants_product_info = any("product info" in msg["content"].lower() for msg in messages)
    wants_requirements = any("requirements" in msg["content"].lower() for msg in messages)
    wants_product_suggestion = any("suggest product" in msg["content"].lower() for msg in messages)

    next_nodes = []
    if wants_customer_info:
        next_nodes.append("customer_info_agent")
    if wants_product_info:
        next_nodes.append("product_info_agent")
    if wants_requirements:
        next_nodes.append("requirements_agent")
    if wants_product_suggestion:
        next_nodes.append("product_suggestion_agent")

    if not next_nodes:
        return Command(update={"messages": messages}, goto="END")

    # Return first next node for simplicity
    return Command(update={"messages": messages}, goto=next_nodes[0])

# --- customer_info_agent node ---
def update_customer_info(state: MessagesState) -> MessagesState:
    \"\"\"Agent for updating customer information.\"\"\"
    messages = state.get("messages", [])

    customer_info: Dict[str, Any] = {}

    def extract_customer_info_from_message(message: Dict[str, Any]) -> None:
        keys_of_interest = ["name", "contact", "email", "phone", "address", "preferences"]
        for key in keys_of_interest:
            if key in message:
                customer_info[key] = message[key]

    for message in messages:
        extract_customer_info_from_message(message)

    if not customer_info:
        return state

    updated_info_message: Message = {
        "role": "system",
        "content": "Updated customer information.",
        **customer_info
    }

    filtered_messages = [
        msg for msg in messages
        if not (msg.get("role") == "system" and "Updated customer information." in msg.get("content", ""))
    ]

    filtered_messages.append(updated_info_message)

    return {"messages": filtered_messages}

# --- product_info_agent node ---
@tool
def get_product_details(product_id: int) -> dict:
    \"\"\"
    Fetches details about a specified product from a predefined database.

    Args:
        product_id (int): The identifier of the product to retrieve details for.

    Returns:
        dict: A dictionary containing product details such as name, price, description, and availability.
    \"\"\"

    # Set up environment variables for Cassandra connection
    os.environ['CASSANDRA_CONTACT_POINTS'] = 'your_contact_points'
    os.environ['CASSANDRA_USERNAME'] = 'your_username'
    os.environ['CASSANDRA_PASSWORD'] = 'your_password'
    os.environ['CASSANDRA_KEYSPACE'] = 'your_keyspace'

    # Initialize Cassandra connection
    db = CassandraDatabase()

    # Query to fetch product details
    query = f"SELECT * FROM products WHERE product_id = {product_id};"
    result = db.execute(query)

    # Assuming result is a single row with product details
    product_details = {
        "product_id": result.product_id,
        "name": result.name,
        "price": result.price,
        "description": result.description,
        "availability": result.availability
    }

    return product_details

@tool
def get_product_reviews(product_id: str, database: str) -> list:
    \"\"\"
    Retrieves reviews for a specified product from a predefined database.

    Args:
        product_id (str): The ID of the product for which reviews are to be retrieved.
        database (str): The name of the database to query.

    Returns:
        list: A list of dictionaries containing reviews for the specified product.
    \"\"\"

    # Set up environment variables for Cassandra connection
    os.environ['CASSANDRA_CONTACT_POINTS'] = 'your_contact_points'
    os.environ['CASSANDRA_USERNAME'] = 'your_username'
    os.environ['CASSANDRA_PASSWORD'] = 'your_password'
    os.environ['CASSANDRA_KEYSPACE'] = database
    # Create a CassandraDatabase instance
    db = CassandraDatabase()

    # Query to retrieve reviews for the specified product
    query = f"SELECT * FROM reviews WHERE product_id = '{product_id}';"
    reviews = db.execute(query)

    # Convert the result to a list of dictionaries
    reviews_list = [dict(review) for review in reviews]

    return reviews_list

tools = [get_product_details, get_product_reviews]

model_with_tools = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
).bind_tools(tools)

tool_node = ToolNode(tools)

def provide_product_info(state: MessagesState) -> MessagesState:
    messages = state.get("messages", [])
    system_prompt = SystemMessage(content="You are an agent specialized in providing detailed and accurate product information. Use the available tools to fetch product details and reviews as needed.")
    response = model_with_tools.invoke([system_prompt] + [HumanMessage(content=msg["content"]) if msg.get("role") == "user" else AIMessage(content=msg["content"]) for msg in messages])
    return {"messages": messages + [{"role": "assistant", "content": response.content}]}

# --- requirements_agent node ---
def ask_customer_requirements(state: MessagesState) -> MessagesState:
    initial_prompt = (
        "Hello! I'm here to help understand your requirements clearly. "
        "Could you please tell me about your needs, preferences, constraints, "
        "and any specific goals you have? Feel free to provide as much detail as possible."
    )
    updated_messages = state.get("messages", []).copy()
    updated_messages.append({"role": "agent", "content": initial_prompt})
    return {"messages": updated_messages}

# --- product_suggestion_agent node ---
def suggest_products(state: MessagesState) -> MessagesState:
    messages = state.get("messages", [])

    conversation_text = ""
    for msg in messages:
        role = msg.get("role", "user")
        content = msg.get("content", "")
        if role == "user":
            conversation_text += f"Customer: {content}\n"
        elif role == "assistant":
            conversation_text += f"Sales assistant: {content}\n"
        else:
            conversation_text += f"{role.capitalize()}: {content}\n"

    prompt = (
        "You are an intelligent product recommendation agent. You will be given a conversation history between a customer and a sales assistant. "
        "Your task is to analyze the customer's messages and suggest relevant products that best meet their needs and preferences. "
        "Respond in a friendly and helpful manner, providing clear product suggestions based on the conversation context. "
        "If the customer has not provided enough information, ask clarifying questions to better understand their requirements before making recommendations.\n\n"
        f"Conversation history:\n{conversation_text}\n"
        "Please provide your product suggestions or questions to the customer to assist them."
    )

    customer_text = " ".join(msg["content"].lower() for msg in messages if msg.get("role") == "user")

    if any(keyword in customer_text for keyword in ["phone", "smartphone", "mobile"]):
        response_text = (
            "Based on your interest in smartphones, I recommend checking out the latest XYZ Phone with excellent camera quality and battery life. "
            "Would you like to know about pricing or available colors?"
        )
    elif any(keyword in customer_text for keyword in ["laptop", "notebook", "computer"]):
        response_text = (
            "I see you're interested in laptops. The ABC Ultrabook offers great performance and portability. "
            "Do you have a preferred brand or budget in mind?"
        )
    elif any(keyword in customer_text for keyword in ["headphones", "earbuds", "audio"]):
        response_text = (
            "For audio devices, the DEF Wireless Earbuds provide superb sound quality and noise cancellation. "
            "Would you like to hear about their features or pricing?"
        )
    else:
        response_text = (
            "Could you please tell me more about what kind of products you are interested in? "
            "This will help me suggest the best options for you."
        )

    updated_messages = messages + [{"role": "assistant", "content": response_text}]
    return {"messages": updated_messages}

# --- Build the workflow graph ---
graph = StateGraph(MessagesState)

# Add nodes
graph.add_node("supervisor_agent", supervisor_agent)
graph.add_node("customer_info_agent", update_customer_info)
graph.add_node("product_info_agent", provide_product_info)
graph.add_node("requirements_agent", ask_customer_requirements)
graph.add_node("product_suggestion_agent", suggest_products)
graph.add_node("tools", tool_node)

# Add edges according to the workflow
graph.add_edge(START, "supervisor_agent")


# Return edges from agents back to supervisor_agent
graph.add_edge("customer_info_agent", "supervisor_agent")
graph.add_edge("product_info_agent", "supervisor_agent")
graph.add_edge("requirements_agent", "supervisor_agent")
graph.add_edge("product_suggestion_agent", "supervisor_agent")

# Also add edges for product_info_agent to tools and back to product_info_agent for tool usage
graph.add_edge("product_info_agent", "tools")
graph.add_edge("tools", "product_info_agent")

# Compile the graph with InMemoryCheckpointer
final_app = graph.compile()
"""

In [3]:
from model_factory import get_model
llm = get_model()

In [6]:
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate

SYS_PROMPT = """
You are given a code, look for any methods with 'tool' decorator and modify them to have mock implementation.

Return the modified code in the same format as the input code.
"""


response = llm.invoke([SystemMessage(content=SYS_PROMPT), HumanMessage(content=str_code)])

In [7]:
response.pretty_print()

================================== Ai Message ==================================


from typing import TypedDict, List, Dict, Any, Literal
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_openai import ChatOpenAI
from langgraph.types import Command
import os
from langchain_community.utilities.cassandra_database import CassandraDatabase


# Define Message and MessagesState types
class Message(TypedDict, total=False):
    role: str
    sender: str
    content: str
    timestamp: str
    escalated: bool
    # Additional fields can be added as needed, e.g. 'status', 'agent_id', etc.

# --- supervisor_agent node ---
def supervisor_agent(state: MessagesState) -> Command[Literal["customer_info_agent", "product_info_agent", "requirements_agent", "product_suggestion_agent", "__end__"]]:
    """A supervisor managin